In [2]:
import time

import pandas as pd
import tushare as ts

type_max = {'北交所': 29.8, '创业板': 19.8}
# 连续涨停数
grow_up_count = 6

ts.set_token('your token here')
# pro = ts.pro_api()
tu_token = 'fba0f69c02f5e4399b8dd63ae895ca69b7a55122a545c6f0f2c2bef3'
pro = ts.pro_api(tu_token)
start_date = '20210117'
end_date = '20220228'


def get_single_grow(ts_code, name, market):
    """
    计算从一个非涨停开始，连续两个涨停之后第三天的涨跌幅
    :param ts_code:
    :return:
    """
    df = pro.daily(ts_code=ts_code, start_date=start_date, end_date=end_date)
    df = df.iloc[::-1] #翻转，否则会按时间倒叙
    # df_last = df.shift(-1)
    df_day = pd.concat([df.ts_code, df.trade_date, df.close, df.pct_chg], axis=1)
    df_day.columns = ['ts_code', 'trade_date', 'close', 'grow']
    # # print("max : " , type_max.get(market))
    market_max = type_max.get(market)
    if None is market_max:
        market_max = 9.8
    list_industry = {'股票代码': [], '股票名称': [], '开始': [], '结果': [], '市场': []}
    df_res = pd.DataFrame(list_industry)
    tmp_start = start_date
    count = 0
    for index, row in df_day.iterrows():
        if count == grow_up_count:
            tmp_start = row['trade_date']
            list_industry['股票代码'].append(row['ts_code'])
            list_industry['股票名称'].append(name)
            list_industry['开始'].append(tmp_start)
            list_industry['结果'].append(row['grow'])
            list_industry['市场'].append(market)

        if row['grow'] < market_max:
            tmp_start = row['trade_date']
            count = 0
        else:
            count += 1
    df_res = pd.DataFrame(list_industry)
    # print("%s : %s : %d" % (ts_code, name, df_res.shape[0]))
    return df_res


def get_single_data():
    df_all = pro.query('stock_basic', exchange='', list_status='L', fields='ts_code,name,market')
    list_industry = {'股票代码': [], '股票名称': [], '开始': [], '结果': [], '市场': []}
    df_res = pd.DataFrame(list_industry)
    size = df_all.shape[0]
    count = 1
    for index, row in df_all.iterrows():
        print("%d : %s , %s  %d / %d" % (grow_up_count, row['ts_code'], row['name'], index, size))
        df_tmp = get_single_grow(row['ts_code'], row['name'], row['market'])
        df_res = df_res.append(df_tmp)
        count += 1
        # if count > 3:
        #     break
    tag = "%s至今开始连续%d个涨停之后第%d天的走势" % (start_date, grow_up_count, (grow_up_count + 1))
    df_res.to_csv('./res/%s.csv' % tag)
    print("*" * 50)
    print(tag)
    str_res = "总共出现： %d 次\n涨 %d 次， 跌 %d 次， 平 %d 次\n平均值： %f" % (df_res.shape[0],
                                                               df_res.loc[df_res["结果"] > 0].shape[0],
                                                               df_res.loc[df_res["结果"] < 0].shape[0],
                                                               df_res.loc[df_res["结果"] == 0].shape[0],
                                                               df_res['结果'].sum() / df_res.shape[0])
    print(str_res)
    file_path = "./res/%s.txt" % tag
    with open(file_path, "w+") as f:
        f.write(str_res)


for i in range(11, 15):
    print("\n")
    print(time.asctime(time.localtime(time.time())))
    grow_up_count = i
    get_single_data()




Mon Feb  7 09:25:56 2022
11 : 000001.SZ , 平安银行  0 / 4714
11 : 000002.SZ , 万科A  1 / 4714
11 : 000004.SZ , 国华网安  2 / 4714
11 : 000005.SZ , ST星源  3 / 4714
11 : 000006.SZ , 深振业A  4 / 4714
11 : 000007.SZ , *ST全新  5 / 4714
11 : 000008.SZ , 神州高铁  6 / 4714
11 : 000009.SZ , 中国宝安  7 / 4714
11 : 000010.SZ , 美丽生态  8 / 4714
11 : 000011.SZ , 深物业A  9 / 4714
11 : 000012.SZ , 南玻A  10 / 4714
11 : 000014.SZ , 沙河股份  11 / 4714
11 : 000016.SZ , 深康佳A  12 / 4714
11 : 000017.SZ , 深中华A  13 / 4714
11 : 000019.SZ , 深粮控股  14 / 4714
11 : 000020.SZ , 深华发A  15 / 4714
11 : 000021.SZ , 深科技  16 / 4714
11 : 000023.SZ , 深天地A  17 / 4714
11 : 000025.SZ , 特力A  18 / 4714
11 : 000026.SZ , 飞亚达  19 / 4714
11 : 000027.SZ , 深圳能源  20 / 4714
11 : 000028.SZ , 国药一致  21 / 4714
11 : 000029.SZ , 深深房A  22 / 4714
11 : 000030.SZ , 富奥股份  23 / 4714
11 : 000031.SZ , 大悦城  24 / 4714
11 : 000032.SZ , 深桑达A  25 / 4714
11 : 000034.SZ , 神州数码  26 / 4714
11 : 000035.SZ , 中国天楹  27 / 4714
11 : 000036.SZ , 华联控股  28 / 4714
11 : 000037.SZ , 深南电A  29 / 471

KeyboardInterrupt: 